In [1]:
from utils.flax_utils import restore_agent
import glob, tqdm, wandb, os, json, random, time, jax
from absl import app, flags
from ml_collections import config_flags
from log_utils import setup_wandb, get_exp_name, get_flag_dict, CsvLogger

from envs.env_utils import make_env_and_datasets
from envs.ogbench_utils import make_ogbench_env_and_datasets
from envs.robomimic_utils import is_robomimic_env

from utils.flax_utils import save_agent
from utils.datasets import Dataset, ReplayBuffer

from evaluation import evaluate
from agents import agents
import numpy as np

ROBOMIMIC WARNING(
    No private macro file found!
    It is recommended to use a private macro file
    To setup, run: python /home/v-hokyunim/miniconda3/envs/debs/lib/python3.10/site-packages/robomimic/scripts/setup_macros.py
)


In [2]:
# handle dataset
dataset_proportion = 1.0
def process_train_dataset(ds):
    """
    Process the train dataset to 
        - handle dataset proportion
        - handle sparse reward
        - convert to action chunked dataset
    """

    ds = Dataset.create(**ds)
    if dataset_proportion < 1.0:
        new_size = int(len(ds['masks']) * dataset_proportion)
        ds = Dataset.create(
            **{k: v[:new_size] for k, v in ds.items()}
        )

    return ds

In [3]:
ds_name = 'cube-double-play-singletask-task1-v0'
env, eval_env, train_dataset, val_dataset = make_env_and_datasets(ds_name)

train_dataset = process_train_dataset(train_dataset)
example_batch = train_dataset.sample(())

/home/v-hokyunim/miniconda3/envs/debs/lib/python3.10/site-packages/gymnasium/spaces/box.py:235: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
/home/v-hokyunim/miniconda3/envs/debs/lib/python3.10/site-packages/gymnasium/spaces/box.py:305: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


In [4]:
FLAGS = flags.FLAGS

flags.DEFINE_string('run_group', 'Debug', 'Run group.')
flags.DEFINE_integer('seed', 0, 'Random seed.')
flags.DEFINE_string('env_name', 'cube-triple-play-singletask-task2-v0', 'Environment (dataset) name.')
flags.DEFINE_string('save_dir', 'exp/', 'Save directory.')

flags.DEFINE_integer('offline_steps', 1000000, 'Number of online steps.')
flags.DEFINE_integer('online_steps', 1000000, 'Number of online steps.')
flags.DEFINE_integer('buffer_size', 2000000, 'Replay buffer size.')
flags.DEFINE_integer('log_interval', 5000, 'Logging interval.')
flags.DEFINE_integer('eval_interval', 100000, 'Evaluation interval.')
flags.DEFINE_integer('save_interval', -1, 'Save interval.')
flags.DEFINE_integer('start_training', 5000, 'when does training start')

flags.DEFINE_integer('utd_ratio', 1, "update to data ratio")

flags.DEFINE_float('discount', 0.99, 'discount factor')

flags.DEFINE_integer('eval_episodes', 50, 'Number of evaluation episodes.')
flags.DEFINE_integer('video_episodes', 0, 'Number of video episodes for each task.')
flags.DEFINE_integer('video_frame_skip', 3, 'Frame skip for videos.')

config_flags.DEFINE_config_file('agent', 'agents/debs.py', lock_config=False)

flags.DEFINE_float('dataset_proportion', 1.0, "Proportion of the dataset to use")
flags.DEFINE_integer('dataset_replace_interval', 1000, 'Dataset replace interval, used for large datasets because of memory constraints')
flags.DEFINE_string('ogbench_dataset_dir', None, 'OGBench dataset directory')

flags.DEFINE_integer('horizon_length', 5, 'action chunking length.')
flags.DEFINE_bool('sparse', False, "make the task sparse reward")

flags.DEFINE_bool('save_all_online_states', False, "save all trajectories to npy")

In [5]:
import json
import ml_collections

def load_config_from_json(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    # 딕셔너리를 ConfigDict로 변환 (점 표기법 사용 가능 및 Type safety)
    config = ml_collections.ConfigDict(data)
    return config

path = '/home/v-hokyunim/Offline/debs/exp/debs/reproduce/cube-double-play-singletask-task1-v0/sd00020251127_134918'
flag_config = load_config_from_json(f'{path}/flags.json')
config = flag_config.agent
config["horizon_length"] = flag_config.horizon_length

In [6]:
agent_class = agents['debs']
agent = agent_class.create(
    0,
    example_batch['observations'],
    example_batch['actions'],
    config,
)

In [7]:
agent = restore_agent(agent, path, 1000000)

Restored from /home/v-hokyunim/Offline/debs/exp/debs/reproduce/cube-double-play-singletask-task1-v0/sd00020251127_134918/params_1000000.pkl


In [ ]:
train_dataset['observations']

(1000000, 37)

In [ ]:
import matplotlib.pyplot as plt

for i in range(10):
    obs_sample = train_dataset['observations'][i*1000:(i+1)*1000]
    rew_sample = train_Dataset['rewards'][i*1000:(i+1)*1000]
    current_zs = agent.network.select('critic')(obs_sample, actions=None)
    np_zs = np.array(current_zs)

    

In [15]:
np.array(current_zs)

array([[-199.41269, -199.38884, -199.39699, ..., -199.4111 , -199.39836,
        -199.39636],
       [-199.41269, -199.38884, -199.39699, ..., -199.4111 , -199.39836,
        -199.39636],
       [-199.41269, -199.38884, -199.39699, ..., -199.4111 , -199.39836,
        -199.39636],
       ...,
       [-199.41249, -199.3888 , -199.39676, ..., -199.41104, -199.39821,
        -199.3962 ],
       [-199.41249, -199.3888 , -199.39676, ..., -199.41104, -199.39821,
        -199.3962 ],
       [-199.41249, -199.3888 , -199.39676, ..., -199.41104, -199.39821,
        -199.3962 ]], shape=(1000, 51), dtype=float32)